<a href="https://colab.research.google.com/github/PhazeRoOman/DeepDTA/blob/Menna's-branch/new_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.io import loadmat
import tensorflow as tf, keras
from keras import layers
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import MeanSquaredError
from keras.callbacks import EarlyStopping,LearningRateScheduler
from google.colab import drive
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers.regularization.spatial_dropout3d import Dropout
from keras.layers import Input, Dense, Average, Concatenate, Conv1D, GlobalAveragePooling1D, Embedding
from keras.models import Model

In [2]:
smiles_max_len = 100
protein_max_len = 1000


smiles_dict = {"#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2,
                "1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6,
                "9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43,
                "D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13,
                "O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51,
                "V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\\": 20, "a": 55, "c": 56,
                "b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60,
                "l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64}

smiles_dict_len = 64

In [3]:
def attention_protein(query, key, value):

    scores = tf.matmul(query, tf.transpose(key, (0,2,1)))
    attention_weights = tf.nn.softmax(scores, axis=-1)
    attended_values = tf.matmul(attention_weights, value)

    return attention_weights, attended_values

In [4]:
Drug_input = Input(shape=(smiles_max_len,), dtype='int32',name='drug_input')
encode_smiles = Embedding(input_dim=smiles_dict_len+1, output_dim = 100, input_length=smiles_max_len,name='smiles_embedding')(Drug_input)
encode_smiles = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=1, name='conv1_smiles')(encode_smiles)
# encode_smiles = MaxPooling1D(2)(encode_smiles)  # test it is effectuve or not
encode_smiles = Conv1D(filters=100*2, kernel_size=3,  activation='relu', padding='valid',  strides=1, name='conv2_smiles')(encode_smiles)
#encode_smiles = MaxPooling1D(2)(encode_smiles)
encode_smiles = Conv1D(filters=100*3, kernel_size=3,  activation='relu', padding='valid',  strides=1, name='conv3_smiles')(encode_smiles)
encode_smiles = GlobalAveragePooling1D()(encode_smiles)

In [5]:
Protein_input = Input(shape=(protein_max_len,200),name='protein_input')
conv_prot_1 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv1_prot')(Protein_input)
conv_prot_2 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv2_prot')(conv_prot_1)
conv_prot_3 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv3_prot')(conv_prot_2)
conv_prot_4 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv4_prot')(conv_prot_3)
conv_prot_5 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv5_prot')(conv_prot_4)
conv_prot_6 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv6_prot')(conv_prot_5)
conv_prot_7 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='same',  strides=1, name='conv7_prot')(conv_prot_6)


In [6]:
attention_weights1, attended_values1 = attention_protein(conv_prot_2,conv_prot_2,conv_prot_2)
attention_weights2, attended_values2 = attention_protein(conv_prot_2,conv_prot_4,conv_prot_4)
attention_weights3, attended_values3 = attention_protein(conv_prot_2,conv_prot_7,conv_prot_7)
attention_weights4, attended_values4 = attention_protein(conv_prot_4,conv_prot_4,conv_prot_4)
attention_weights5, attended_values5 = attention_protein(conv_prot_4,conv_prot_2,conv_prot_2)
attention_weights6, attended_values6 = attention_protein(conv_prot_4,conv_prot_7,conv_prot_7)
attention_weights7, attended_values7 = attention_protein(conv_prot_7,conv_prot_4,conv_prot_4)
attention_weights8, attended_values8 = attention_protein(conv_prot_7,conv_prot_2,conv_prot_2)
attention_weights9, attended_values9 = attention_protein(conv_prot_7,conv_prot_7,conv_prot_7)

In [7]:
attention_weights = Average()([attention_weights1,attention_weights2,attention_weights3,attention_weights4,attention_weights5,
           attention_weights6,attention_weights7,attention_weights8,attention_weights9])
attended_values = Average()([attended_values1,attended_values2,attended_values3,attended_values4,attended_values5,
                             attended_values6,attended_values7,attended_values8,attended_values9])
conv_prot_att_weights_1 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=2, name='conv_prot_att_weights_1')(attention_weights)
conv_prot_att_weights_2 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=2, name='conv_prot_att_weights_2')(conv_prot_att_weights_1)
conv_prot_att_weights_3 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=1, name='conv_prot_att_weights_3')(conv_prot_att_weights_2)

conv_prot_att_value_1 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=2, name='conv_prot_att_value_1')(attended_values)
conv_prot_att_value_2 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=2, name='conv_prot_att_value_2')(conv_prot_att_value_1)
conv_prot_att_value_3 = Conv1D(filters=100, kernel_size=3,  activation='relu', padding='valid',  strides=1, name='conv_prot_att_value_3')(conv_prot_att_value_2)

In [8]:
prot_att_weights = GlobalAveragePooling1D()(conv_prot_att_weights_3)
prot_att_value = GlobalAveragePooling1D()(conv_prot_att_value_3)

prot_out = Concatenate()([prot_att_weights, prot_att_value])

encode_interaction = Concatenate()([prot_out, encode_smiles])

FC1 = Dense(1024, activation='relu', name='dense1')(encode_interaction)
FC2 = Dropout(0.1)(FC1)
FC2 = Dense(1024, activation='relu', name='dense2')(FC2)
FC2 = Dropout(0.1)(FC2)
FC2 = Dense(512, activation='relu', name='dense3')(FC2)
# And add a logistic regression on top
predictions = Dense(1, kernel_initializer='normal', name='dense4')(FC2) # if you want train model for active/inactive set activation='sigmoid'

model = Model(inputs=[Drug_input, Protein_input], outputs=[predictions])

print(model.summary())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 protein_input (InputLayer)     [(None, 1000, 200)]  0           []                               
                                                                                                  
 conv1_prot (Conv1D)            (None, 1000, 100)    60100       ['protein_input[0][0]']          
                                                                                                  
 conv2_prot (Conv1D)            (None, 1000, 100)    30100       ['conv1_prot[0][0]']             
                                                                                                  
 conv3_prot (Conv1D)            (None, 1000, 100)    30100       ['conv2_prot[0][0]']             
                                                                                              